. goals made || attk score   
. goals allowed || defeense score   
. games score (lose-0 | tie-1 | win-3)    

. head 2 head (season | last2 seasons | historical)   

. LAST X GAMES

In [194]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [195]:
df = pd.read_csv('../data/raw/dataset.csv')
df

,fixture_id,date,venue_id,venue,status,game_time,extra_time,home_team,home_team_id,away_team,...,away_winner,home_ht_score,away_ht_score,home_ft_score,away_ft_score,home_extra_score,away_extra_score,referee,league_season,home_result
0,720746,2021-08-13 19:00:00+00:00,1497,Estadio de Mestalla,Match Finished,90,NaN,Valencia,532,Getafe,...,False,1,0,1,0,NaN,NaN,Jesús Gil,2021,WIN
1,720743,2021-08-14 17:30:00+00:00,11915,Estadio Nuevo Mirandilla,Match Finished,90,NaN,Cadiz,724,Levante,...,NaN,0,1,1,1,NaN,NaN,Santiago Jaime,2021,DRAW
2,720742,2021-08-14 17:30:00+00:00,1484,Iberostar Estadi,Match Finished,90,NaN,Mallorca,798,Real Betis,...,NaN,1,0,1,1,NaN,NaN,Antonio Mateu,2021,DRAW
3,720739,2021-08-14 20:00:00+00:00,1470,Estadio de Mendizorroza,Match Finished,90,NaN,Alaves,542,Real Madrid,...,True,0,0,1,4,NaN,NaN,César Soto,2021,LOSS
4,720744,2021-08-14 20:00:00+00:00,1486,Estadio El Sadar,Match Finished,90,NaN,Osasuna,727,Espanyol,...,NaN,0,0,0,0,NaN,NaN,Miguel Ortiz,2021,DRAW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,1038330,2024-05-25 19:00:00+00:00,1456,Estadio Santiago Bernabéu,Match Finished,90,NaN,Real Madrid,541,Real Betis,...,NaN,0,0,0,0,NaN,NaN,"Isidro Diaz de Mera Escuderos, Spain",2023,DRAW
1136,1038324,2024-05-26 12:00:00+00:00,20422,Estadio Coliseum,Match Finished,90,NaN,Getafe,546,Mallorca,...,True,0,0,1,2,NaN,NaN,"Victor Garcia Verdura, Spain",2023,LOSS
1137,1038331,2024-05-26 14:15:00+00:00,1481,Estadio de Gran Canaria,Match Finished,90,NaN,Las Palmas,534,Alaves,...,NaN,0,0,1,1,NaN,NaN,"Francisco Hernandez Maeso, Spain",2023,DRAW
1138,1038323,2024-05-26 14:15:00+00:00,1467,Abanca-Balaídos,Match Finished,90,NaN,Celta Vigo,538,Valencia,...,NaN,0,1,2,2,NaN,NaN,"Miguel Angel Ortiz Arias, Spain",2023,DRAW


In [196]:
# df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%d-%m-%Y')

In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fixture_id        1140 non-null   int64  
 1   date              1140 non-null   object 
 2   venue_id          1140 non-null   int64  
 3   venue             1140 non-null   object 
 4   status            1140 non-null   object 
 5   game_time         1140 non-null   int64  
 6   extra_time        0 non-null      float64
 7   home_team         1140 non-null   object 
 8   home_team_id      1140 non-null   int64  
 9   away_team         1140 non-null   object 
 10  away_team_id      1140 non-null   int64  
 11  home_score        1140 non-null   int64  
 12  away_score        1140 non-null   int64  
 13  home_winner       833 non-null    object 
 14  away_winner       833 non-null    object 
 15  home_ht_score     1140 non-null   int64  
 16  away_ht_score     1140 non-null   int64  


In [198]:
df['status'].value_counts()

status
Match Finished    1140
Name: count, dtype: int64

In [199]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

C:\Users\danie\AppData\Local\Temp\ipykernel_21300\241827952.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['date'])


In [200]:
label_map = {'LOSS': 0, 'DRAW': 1, 'WIN': 2}
df['home_result'] = df['home_result'].map(label_map)

In [201]:
df

,fixture_id,date,venue_id,venue,status,game_time,extra_time,home_team,home_team_id,away_team,...,away_winner,home_ht_score,away_ht_score,home_ft_score,away_ft_score,home_extra_score,away_extra_score,referee,league_season,home_result
0,720746,2021-08-13,1497,Estadio de Mestalla,Match Finished,90,NaN,Valencia,532,Getafe,...,False,1,0,1,0,NaN,NaN,Jesús Gil,2021,2
1,720743,2021-08-14,11915,Estadio Nuevo Mirandilla,Match Finished,90,NaN,Cadiz,724,Levante,...,NaN,0,1,1,1,NaN,NaN,Santiago Jaime,2021,1
2,720742,2021-08-14,1484,Iberostar Estadi,Match Finished,90,NaN,Mallorca,798,Real Betis,...,NaN,1,0,1,1,NaN,NaN,Antonio Mateu,2021,1
3,720739,2021-08-14,1470,Estadio de Mendizorroza,Match Finished,90,NaN,Alaves,542,Real Madrid,...,True,0,0,1,4,NaN,NaN,César Soto,2021,0
4,720744,2021-08-14,1486,Estadio El Sadar,Match Finished,90,NaN,Osasuna,727,Espanyol,...,NaN,0,0,0,0,NaN,NaN,Miguel Ortiz,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,1038328,2024-05-25,1491,Reale Arena,Match Finished,90,NaN,Real Sociedad,548,Atletico Madrid,...,True,0,1,0,2,NaN,NaN,"Jose Maria Sanchez Martinez, Spain",2023,0
1137,1038331,2024-05-26,1481,Estadio de Gran Canaria,Match Finished,90,NaN,Las Palmas,534,Alaves,...,NaN,0,0,1,1,NaN,NaN,"Francisco Hernandez Maeso, Spain",2023,1
1138,1038323,2024-05-26,1467,Abanca-Balaídos,Match Finished,90,NaN,Celta Vigo,538,Valencia,...,NaN,0,1,2,2,NaN,NaN,"Miguel Angel Ortiz Arias, Spain",2023,1
1136,1038324,2024-05-26,20422,Estadio Coliseum,Match Finished,90,NaN,Getafe,546,Mallorca,...,True,0,0,1,2,NaN,NaN,"Victor Garcia Verdura, Spain",2023,0


## **DROP FEATURES**

In [202]:
df.drop(columns=['venue_id','venue','status','game_time','extra_time','home_winner','away_winner','home_ht_score','home_ft_score','away_ht_score','away_ft_score','home_extra_score','home_extra_score','away_extra_score','away_extra_score','referee'], inplace=True)
df

,fixture_id,date,home_team,home_team_id,away_team,away_team_id,home_score,away_score,league_season,home_result
0,720746,2021-08-13,Valencia,532,Getafe,546,1,0,2021,2
1,720743,2021-08-14,Cadiz,724,Levante,539,1,1,2021,1
2,720742,2021-08-14,Mallorca,798,Real Betis,543,1,1,2021,1
3,720739,2021-08-14,Alaves,542,Real Madrid,541,1,4,2021,0
4,720744,2021-08-14,Osasuna,727,Espanyol,540,0,0,2021,1
...,...,...,...,...,...,...,...,...,...,...
1132,1038328,2024-05-25,Real Sociedad,548,Atletico Madrid,530,0,2,2023,0
1137,1038331,2024-05-26,Las Palmas,534,Alaves,542,1,1,2023,1
1138,1038323,2024-05-26,Celta Vigo,538,Valencia,532,2,2,2023,1
1136,1038324,2024-05-26,Getafe,546,Mallorca,798,1,2,2023,0


## **RESTRUCTURE DATA**

In [203]:
home_df = df[['fixture_id', 'league_season','date', 'home_team', 'home_score','away_score']].copy()
home_df.columns = ['fixture_id', 'season', 'date','team', 'goals_for', 'goals_against']
home_df['location'] = 'home'


In [204]:
away_df = df[['fixture_id','league_season', 'date','away_team', 'away_score','home_score']].copy()
away_df.columns = ['fixture_id', 'season','date', 'team', 'goals_for', 'goals_against']
away_df['location'] = 'away'

In [205]:
long_df = pd.concat([home_df, away_df])
long_df = long_df.sort_values(['team', 'date']).reset_index(drop=True)

long_df['team'].unique()


array(['Alaves', 'Almeria', 'Athletic Club', 'Atletico Madrid',
       'Barcelona', 'Cadiz', 'Celta Vigo', 'Elche', 'Espanyol', 'Getafe',
       'Girona', 'Granada CF', 'Las Palmas', 'Levante', 'Mallorca',
       'Osasuna', 'Rayo Vallecano', 'Real Betis', 'Real Madrid',
       'Real Sociedad', 'Sevilla', 'Valencia', 'Valladolid', 'Villarreal'],
      dtype=object)

## **AVG GOALS**

In [206]:
for N in [3, 5, 10]:

    long_df[f'avg_goals_last_{N}'] = (long_df.groupby('team')['goals_for'].transform(lambda x: x.shift(1).rolling(window=N, min_periods=1).mean().round(3)))

    long_df[f'avg_goals_against_last_{N}'] = (long_df.groupby('team')['goals_against'].transform(lambda x: x.shift(1).rolling(window=N, min_periods=1).mean().round(3)))


# long_df['avg_goals_season'] = long_df.groupby(['team','season'])['goals_made'].transform('mean')

long_df['avg_goals_season'] = (
    long_df.groupby(['team', 'season'])['goals_for'].transform(lambda x: x.shift(1).expanding(min_periods=1).mean().round(3))
)
long_df['avg_goals_against_season'] = (
    long_df.groupby(['team', 'season'])['goals_against'].transform(lambda x: x.shift(1).expanding(min_periods=1).mean().round(3))
)


long_df.fillna(0, inplace=True)


long_df['avg_scoring_last_3'] = (long_df['avg_goals_last_3'] - long_df['avg_goals_against_last_3']).round(3)
long_df['avg_scoring_last_5'] = (long_df['avg_goals_last_5'] - long_df['avg_goals_against_last_5']).round(3)
long_df['avg_scoring_10'] = (long_df['avg_goals_last_10'] - long_df['avg_goals_against_last_10']).round(3)
long_df['avg_scoring_season'] = (long_df['avg_goals_season'] - long_df['avg_goals_against_season']).round(3)



long_df

,fixture_id,season,date,team,goals_for,goals_against,location,avg_goals_last_3,avg_goals_against_last_3,avg_goals_last_5,avg_goals_against_last_5,avg_goals_last_10,avg_goals_against_last_10,avg_goals_season,avg_goals_against_season,avg_scoring_last_3,avg_scoring_last_5,avg_scoring_10,avg_scoring_season
0,720739,2021,2021-08-14,Alaves,1,4,home,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,720749,2021,2021-08-21,Alaves,0,1,home,1.000,4.000,1.000,4.000,1.000,4.000,1.000,4.000,-3.000,-3.000,-3.000,-3.000
2,720766,2021,2021-08-27,Alaves,0,3,away,0.500,2.500,0.500,2.500,0.500,2.500,0.500,2.500,-2.000,-2.000,-2.000,-2.000
3,720779,2021,2021-09-18,Alaves,0,2,home,0.333,2.667,0.333,2.667,0.333,2.667,0.333,2.667,-2.334,-2.334,-2.334,-2.334
4,720793,2021,2021-09-22,Alaves,0,1,away,0.000,2.000,0.250,2.500,0.250,2.500,0.250,2.500,-2.000,-2.250,-2.250,-2.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,1038282,2023,2024-05-05,Villarreal,2,3,away,2.000,0.667,1.600,0.800,2.100,1.000,1.636,1.667,1.333,0.800,1.100,-0.031
2276,1038300,2023,2024-05-11,Villarreal,3,2,home,2.333,1.333,1.800,1.400,2.200,1.200,1.647,1.706,1.000,0.400,1.000,-0.059
2277,1038303,2023,2024-05-14,Villarreal,1,0,away,2.667,1.667,2.200,1.400,2.400,1.300,1.686,1.714,1.000,0.800,1.100,-0.028
2278,1038321,2023,2024-05-19,Villarreal,4,4,home,2.000,1.667,2.200,1.200,2.200,1.200,1.667,1.667,0.333,1.000,1.000,0.000


## **POINTS**

In [207]:
long_df['points'] = np.where(long_df['goals_for'] > long_df['goals_against'], 3,1)
long_df['points'] = np.where(long_df['goals_for'] < long_df['goals_against'], 0,long_df['points'])

In [208]:
for N in [3, 5, 10]:

    long_df[f'points_last_{N}'] = (long_df.groupby('team')['points'].transform(lambda x: x.shift(1).rolling(window=N, min_periods=1).sum().round(3)))

long_df.fillna(0,inplace=True)

In [209]:
long_df

,fixture_id,season,date,team,goals_for,goals_against,location,avg_goals_last_3,avg_goals_against_last_3,avg_goals_last_5,...,avg_goals_season,avg_goals_against_season,avg_scoring_last_3,avg_scoring_last_5,avg_scoring_10,avg_scoring_season,points,points_last_3,points_last_5,points_last_10
0,720739,2021,2021-08-14,Alaves,1,4,home,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0,0.0,0.0,0.0
1,720749,2021,2021-08-21,Alaves,0,1,home,1.000,4.000,1.000,...,1.000,4.000,-3.000,-3.000,-3.000,-3.000,0,0.0,0.0,0.0
2,720766,2021,2021-08-27,Alaves,0,3,away,0.500,2.500,0.500,...,0.500,2.500,-2.000,-2.000,-2.000,-2.000,0,0.0,0.0,0.0
3,720779,2021,2021-09-18,Alaves,0,2,home,0.333,2.667,0.333,...,0.333,2.667,-2.334,-2.334,-2.334,-2.334,0,0.0,0.0,0.0
4,720793,2021,2021-09-22,Alaves,0,1,away,0.000,2.000,0.250,...,0.250,2.500,-2.000,-2.250,-2.250,-2.250,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,1038282,2023,2024-05-05,Villarreal,2,3,away,2.000,0.667,1.600,...,1.636,1.667,1.333,0.800,1.100,-0.031,0,7.0,10.0,21.0
2276,1038300,2023,2024-05-11,Villarreal,3,2,home,2.333,1.333,1.800,...,1.647,1.706,1.000,0.400,1.000,-0.059,3,6.0,7.0,20.0
2277,1038303,2023,2024-05-14,Villarreal,1,0,away,2.667,1.667,2.200,...,1.686,1.714,1.000,0.800,1.100,-0.028,3,6.0,10.0,22.0
2278,1038321,2023,2024-05-19,Villarreal,4,4,home,2.000,1.667,2.200,...,1.667,1.667,0.333,1.000,1.000,0.000,1,6.0,12.0,22.0


### **JOINING BACK**

In [210]:
home_feats = long_df[long_df['location'] == 'home'].drop(columns=['season','goals_for','goals_against','date','points','location']).rename(columns={
    'team' : 'home_team',
    'avg_goals_last_3': 'home_avg_goals_last_3',
    'avg_goals_against_last_3' : 'home_avg_goals_against_last_3',
    'avg_goals_last_5' : 'home_avg_goals_last_5',
    'avg_goals_against_last_5' : 'home_avg_goals_against_last_5',
    'avg_goals_last_10' : 'home_avg_goals_last_10',
    'avg_goals_against_last_10' : 'home_avg_goals_against_last_10',
    'avg_goals_season' : 'home_avg_goals_season',
    'avg_goals_against_season' : 'home_avg_goals_against_season',
    'avg_scoring_last_3' : 'home_avg_scoring_last_3',
    'avg_scoring_last_5' : 'home_avg_scoring_last_5',
    'avg_scoring_10' : 'home_avg_scoring_last_10',
    'avg_scoring_season' : 'home_avg_scoring_season',
    'points_last_3' : 'home_points_last_3',
    'points_last_5' : 'home_points_last_5',
    'points_last_10' : 'home_points_last_10'
})

In [211]:
away_feats = long_df[long_df['location'] == 'away'].drop(columns=['season','goals_for','goals_against','date','points','location']).rename(columns={
    'team' : 'away_team',
    'avg_goals_last_3': 'away_avg_goals_last_3',
    'avg_goals_against_last_3' : 'away_avg_goals_against_last_3',
    'avg_goals_last_5' : 'away_avg_goals_last_5',
    'avg_goals_against_last_5' : 'away_avg_goals_against_last_5',
    'avg_goals_last_10' : 'away_avg_goals_last_10',
    'avg_goals_against_last_10' : 'away_avg_goals_against_last_10',
    'avg_goals_season' : 'away_avg_goals_season',
    'avg_goals_against_season' : 'away_avg_goals_against_season',
    'avg_scoring_last_3' : 'away_avg_scoring_last_3',
    'avg_scoring_last_5' : 'away_avg_scoring_last_5',
    'avg_scoring_10' : 'away_avg_scoring_last_10',
    'avg_scoring_season' : 'away_avg_scoring_season',
    'points_last_3' : 'away_points_last_3',
    'points_last_5' : 'away_points_last_5',
    'points_last_10' : 'away_points_last_10'
})

In [212]:
df = df.merge(home_feats, on=['fixture_id', 'home_team'], how='left')
df = df.merge(away_feats, on=['fixture_id', 'away_team'], how='left')

In [213]:
df

,fixture_id,date,home_team,home_team_id,away_team,away_team_id,home_score,away_score,league_season,home_result,...,away_avg_goals_against_last_10,away_avg_goals_season,away_avg_goals_against_season,away_avg_scoring_last_3,away_avg_scoring_last_5,away_avg_scoring_last_10,away_avg_scoring_season,away_points_last_3,away_points_last_5,away_points_last_10
0,720746,2021-08-13,Valencia,532,Getafe,546,1,0,2021,2,...,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0
1,720743,2021-08-14,Cadiz,724,Levante,539,1,1,2021,1,...,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0
2,720742,2021-08-14,Mallorca,798,Real Betis,543,1,1,2021,1,...,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0
3,720739,2021-08-14,Alaves,542,Real Madrid,541,1,4,2021,0,...,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0
4,720744,2021-08-14,Osasuna,727,Espanyol,540,0,0,2021,1,...,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,1038328,2024-05-25,Real Sociedad,548,Atletico Madrid,530,0,2,2023,0,...,1.4,1.838,1.162,0.334,0.8,0.0,0.676,6.0,12.0,18.0
1136,1038331,2024-05-26,Las Palmas,534,Alaves,542,1,1,2023,1,...,1.2,0.946,1.216,-1.333,0.0,-0.2,-0.270,4.0,10.0,16.0
1137,1038323,2024-05-26,Celta Vigo,538,Valencia,532,2,2,2023,1,...,1.2,1.027,1.162,-1.000,-1.2,-0.6,-0.135,1.0,1.0,8.0
1138,1038324,2024-05-26,Getafe,546,Mallorca,798,1,2,2023,0,...,0.9,0.838,1.162,0.333,0.0,-0.2,-0.324,5.0,6.0,10.0


In [214]:
df['scoring_diff_last_3'] = df['home_avg_scoring_last_3'] - df['away_avg_scoring_last_3']
df['scoring_diff_last_5'] = df['home_avg_scoring_last_5'] - df['away_avg_scoring_last_5']
df['scoring_diff_last_10'] = df['home_avg_scoring_last_10'] - df['away_avg_scoring_last_10']
df['scoring_diff_season'] = df['home_avg_scoring_season'] - df['away_avg_scoring_season']

df['points_diff_last_3'] = df['home_points_last_3'] - df['away_points_last_3']
df['points_diff_last_5'] = df['home_points_last_5'] - df['away_points_last_5']
df['points_diff_last_10'] = df['home_points_last_10'] - df['away_points_last_10']

df


,fixture_id,date,home_team,home_team_id,away_team,away_team_id,home_score,away_score,league_season,home_result,...,away_points_last_3,away_points_last_5,away_points_last_10,scoring_diff_last_3,scoring_diff_last_5,scoring_diff_last_10,scoring_diff_season,points_diff_last_3,points_diff_last_5,points_diff_last_10
0,720746,2021-08-13,Valencia,532,Getafe,546,1,0,2021,2,...,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0
1,720743,2021-08-14,Cadiz,724,Levante,539,1,1,2021,1,...,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0
2,720742,2021-08-14,Mallorca,798,Real Betis,543,1,1,2021,1,...,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0
3,720739,2021-08-14,Alaves,542,Real Madrid,541,1,4,2021,0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0
4,720744,2021-08-14,Osasuna,727,Espanyol,540,0,0,2021,1,...,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,1038328,2024-05-25,Real Sociedad,548,Atletico Madrid,530,0,2,2023,0,...,6.0,12.0,18.0,-0.001,-0.4,0.6,-0.298,0.0,-3.0,2.0
1136,1038331,2024-05-26,Las Palmas,534,Alaves,542,1,1,2023,1,...,4.0,10.0,16.0,1.000,-1.0,-1.2,-0.108,-2.0,-8.0,-14.0
1137,1038323,2024-05-26,Celta Vigo,538,Valencia,532,2,2,2023,1,...,1.0,1.0,8.0,1.333,1.0,0.4,-0.162,5.0,8.0,8.0
1138,1038324,2024-05-26,Getafe,546,Mallorca,798,1,2,2023,0,...,5.0,6.0,10.0,-2.000,-1.0,-0.4,0.027,-5.0,-3.0,-2.0


## **STREAKS**

## **HEAD 2 HEAD**

In [215]:
# from collections import defaultdict

# history = defaultdict(list)

# history

In [216]:
# from collections import defaultdict

# # Sort matches by date
# df = df.sort_values('date').reset_index(drop=True)

# # Initialize storage
# h2h_features = {
#     'fixture_id': [],
#     'h2h_matches': [],
#     'h2h_home_wins': [],
#     'h2h_away_wins': [],
#     'h2h_draws': [],
#     'h2h_home_goals': [],
#     'h2h_away_goals': [],
# }

# N = 5  # Last N head-to-head matches

# # Historical storage
# history = defaultdict(list)

# # Loop over each match
# for _, row in df.iterrows():
#     home = row['home_team']
#     away = row['away_team']
#     date = row['date']
#     fixture_id = row['fixture_id']

#     # Define a key that ignores home/away for H2H tracking
#     matchup_key = tuple(sorted([home, away]))

#     # Get past N matches before this one
#     past_matches = [m for m in history[matchup_key] if m['date'] < date]
#     past_matches = sorted(past_matches, key=lambda x: x['date'])[-N:]

#     # Extract stats
#     h2h_matches = len(past_matches)
#     home_wins = away_wins = draws = 0
#     home_goals = away_goals = 0

#     for m in past_matches:
#         if m['home'] == home:
#             home_goals += m['home_score']
#             away_goals += m['away_score']
#             if m['home_score'] > m['away_score']:
#                 home_wins += 1
#             elif m['home_score'] < m['away_score']:
#                 away_wins += 1
#             else:
#                 draws += 1
#         else:
#             home_goals += m['away_score']
#             away_goals += m['home_score']
#             if m['away_score'] > m['home_score']:
#                 home_wins += 1
#             elif m['away_score'] < m['home_score']:
#                 away_wins += 1
#             else:
#                 draws += 1

#     # Store features
#     h2h_features['fixture_id'].append(fixture_id)
#     h2h_features['h2h_matches'].append(h2h_matches)
#     h2h_features['h2h_home_wins'].append(home_wins)
#     h2h_features['h2h_away_wins'].append(away_wins)
#     h2h_features['h2h_draws'].append(draws)
#     h2h_features['h2h_home_goals'].append(home_goals)
#     h2h_features['h2h_away_goals'].append(away_goals)

#     # Update history
#     history[matchup_key].append({
#         'date': date,
#         'home': home,
#         'away': away,
#         'home_score': row['home_score'],
#         'away_score': row['away_score']
#     })

# # # Convert to DataFrame and merge
# # h2h_df = pd.DataFrame(h2h_features)
# # df = df.merge(h2h_df, on='fixture_id', how='left')

# history[matchup_key]

In [217]:
df.to_csv('../data/processed.csv', index=False)

In [218]:
df[:-140].to_csv('../data/train.csv', index=False)


In [219]:
df[-140:].to_csv('../data/test.csv', index=False)

In [220]:
df

,fixture_id,date,home_team,home_team_id,away_team,away_team_id,home_score,away_score,league_season,home_result,...,away_points_last_3,away_points_last_5,away_points_last_10,scoring_diff_last_3,scoring_diff_last_5,scoring_diff_last_10,scoring_diff_season,points_diff_last_3,points_diff_last_5,points_diff_last_10
0,720746,2021-08-13,Valencia,532,Getafe,546,1,0,2021,2,...,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0
1,720743,2021-08-14,Cadiz,724,Levante,539,1,1,2021,1,...,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0
2,720742,2021-08-14,Mallorca,798,Real Betis,543,1,1,2021,1,...,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0
3,720739,2021-08-14,Alaves,542,Real Madrid,541,1,4,2021,0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0
4,720744,2021-08-14,Osasuna,727,Espanyol,540,0,0,2021,1,...,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,1038328,2024-05-25,Real Sociedad,548,Atletico Madrid,530,0,2,2023,0,...,6.0,12.0,18.0,-0.001,-0.4,0.6,-0.298,0.0,-3.0,2.0
1136,1038331,2024-05-26,Las Palmas,534,Alaves,542,1,1,2023,1,...,4.0,10.0,16.0,1.000,-1.0,-1.2,-0.108,-2.0,-8.0,-14.0
1137,1038323,2024-05-26,Celta Vigo,538,Valencia,532,2,2,2023,1,...,1.0,1.0,8.0,1.333,1.0,0.4,-0.162,5.0,8.0,8.0
1138,1038324,2024-05-26,Getafe,546,Mallorca,798,1,2,2023,0,...,5.0,6.0,10.0,-2.000,-1.0,-0.4,0.027,-5.0,-3.0,-2.0
